In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Import Libraries

In [ ]:
# import necessary libraries
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import cv2

## Load Dataset

### Train, Test and Validation Datasets

In [ ]:
# take directories of the corresponding datasets
train_data = "chest_xray/train"
test_data = "chest_xray/test"
val_data = "chest_xray/val"

# get the count of respective images in all datasets
train_num_pneumonia = len(os.listdir(os.path.join(train_data, 'PNEUMONIA')))
train_num_normal = len(os.listdir(os.path.join(train_data, 'NORMAL')))

test_num_pneumonia=len(os.listdir(os.path.join(test_data, 'PNEUMONIA')))
test_num_normal=len(os.listdir(os.path.join(test_data, 'NORMAL')))

val_num_pnemonia=len(os.listdir(os.path.join(val_data, 'PNEUMONIA')))
val_num_normal=len(os.listdir(os.path.join(val_data, 'NORMAL')))

# print all the counts of images
print("Train dataset:")
print(f"The number of Pnemonia images in training dataset={train_num_pneumonia}")
print(f"The number of Normal images in training dataset={train_num_normal}")

print("Test dataset:")
print(f"The number of Pnemonia images in testing dataset={test_num_pneumonia}")
print(f"The number of Pnemonia images in testing dataset={test_num_normal}")

print("Validation dataset:")
print(f"The number of Pnemonia images in validation dataset={val_num_pnemonia}")
print(f"The number of Pnemonia images in validation dataset={val_num_normal}")

## Analysis

### Visualize Pneumonia Dataset 

In [ ]:
# plot some of the images of Pneumonia chest X-rays
pneumonia = os.listdir("chest_xray/train/PNEUMONIA")
pneumonia_dataset = "chest_xray/train/PNEUMONIA"

plt.figure(figsize=(20, 10))

for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = plt.imread(os.path.join(pneumonia_dataset, pneumonia[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    
plt.tight_layout()

### Visualize Normal Dataset

In [ ]:
# plot some of the images of Normal chest X-rays
normal = os.listdir("chest_xray/train/NORMAL")
normal_dataset = "chest_xray/train/NORMAL"

plt.figure(figsize=(20, 10))

for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = plt.imread(os.path.join(normal_dataset, normal[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    
plt.tight_layout()

### Number of data points of both classes in training dataset

In [ ]:
# Plot the bar graph for the 2 classes in training dataset
plt.bar(['NORMAL', 'PNEUMONIA'], [train_num_normal, train_num_pneumonia], color=['g', 'r'])
plt.title('Number of Data Points in Each Class')
plt.xlabel('Class')
plt.ylabel('Number of Data Points')
plt.show()

### Number of data points of both classes in testing dataset

In [ ]:
# Plot the bar graph for the 2 classes in training dataset
plt.bar(['NORMAL', 'PNEUMONIA'], [test_num_normal, test_num_pneumonia], color=['g', 'r'])
plt.title('Number of Data Points in each class of testing dataset')
plt.xlabel('Class')
plt.ylabel('Number of Data Points')
plt.show()

### Characteristics of the Image in Dataset

In [ ]:
import glob

# get one of the image of normal chest X-ray and plot 
normal_data = "chest_xray/train/NORMAL"
normal_image_path = glob.glob(normal_data + "/*.jpeg")[0]
sample_image = plt.imread(normal_image_path)
plt.imshow(sample_image, cmap='gray')
plt.colorbar()
plt.title('Raw Chest X Ray Image')

# print the characteristics of the image
print("Image dimensions: {} pixels width x {} pixels height".format(sample_image.shape[0], sample_image.shape[1]))
print("Maximum pixel value: {:.4f}".format(sample_image.max()))
print("Minimum pixel value: {:.4f}".format(sample_image.min()))
print("Mean pixel value: {:.4f}".format(sample_image.mean()))
print("Standard deviation of pixel values: {:.4f}".format(sample_image.std()))

### Histogram of Pixel Intensities in the Image

In [ ]:
# plot the pixel intensities of the sample image
pixel_mean = np.mean(sample_image)
pixel_std = np.std(sample_image)

sns.histplot(sample_image.ravel(), kde=False)
plt.axvline(x=pixel_mean, color='r', linestyle='--', label=f"Pixel Mean {pixel_mean:.4f}")
plt.axvline(x=pixel_std, color='g', linestyle='--', label=f"Pixel Standard Deviation {pixel_std:.4f}")
plt.legend(loc='upper center')
plt.title('Distribution of Pixel Intensities in the Image')
plt.xlabel('Intensity of the pixel')
plt.ylabel('# Image Pixels')

## Data Preprocessing

### Data Preprocessing and Data Augmentation.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator for data prepocessing and data augmentation
image_generator = ImageDataGenerator(
    rotation_range=30,  # increasing the rotation range to 30 degrees
    width_shift_range=0.2,  # increasing the width shift range to 20%
    shear_range=0.2,  # increasing shear range to 20%
    zoom_range=0.2,  # increasing the zoom range to 20%
    samplewise_center=True,  # enabling the samplewise centering
    samplewise_std_normalization=True,  # enabling the samplewise std normalization # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip=False
)

### Train, Test and Validation Image Data Generator

In [ ]:
# load the images to their corresponding variables using different image generators for each
train = image_generator.flow_from_directory(train_data, 
                                            batch_size=8, 
                                            shuffle=True, 
                                            class_mode='binary',
                                            target_size=(180, 180))

test = image_generator.flow_from_directory(test_data, 
                                            batch_size=8, 
                                            shuffle=False, 
                                            class_mode='binary',
                                            target_size=(180, 180))

validation = image_generator.flow_from_directory(val_data, 
                                                batch_size=8, 
                                                shuffle=False, 
                                                class_mode='binary',
                                                target_size=(180, 180))

### Visualize Preprocessed Image 

In [ ]:
# plot the sample augmented image for visualization
sns.set_style('white')
generated_image, label = train.__getitem__(0)
plt.imshow(generated_image[0], cmap='gray')
plt.colorbar()
plt.title('Raw Chest X Ray Image')

print(f"The dimensions of the image are {generated_image.shape[1]} pixels width and {generated_image.shape[2]} pixels height, one single color channel.")
print(f"The maximum pixel value is {generated_image.max():.4f} and the minimum is {generated_image.min():.4f}")
print(f"The mean value of the pixels is {generated_image.mean():.4f} and the standard deviation is {generated_image.std():.4f}")

## Baseline Model

### Convolution Neural Network Model

A CNN model consists of convolutional layers, pooling layers, fully connected layers, and finally more convolutional layers. By introducing filters on the input image to discover patterns and features like edges, curves, and forms, the convolutional layers conduct feature extraction. The pooling layers serve to lower the computational complexity of the model by shrinking the size of the feature maps produced by the convolutional layers. 


The final classification decision is subsequently made by one or more fully connected layers using the output of the final convolutional and pooling layers, which has been flattened and connected. The retrieved features are used by the fully connected layers to determine whether or not pneumonia is present in the chest X-ray pictures.

#### Calculate class weights

In [ ]:
# Calculate class weights and print them
class_0_wgt = train_num_pneumonia / (train_num_normal + train_num_pneumonia)
class_1_wgt = train_num_normal / (train_num_normal + train_num_pneumonia)

class_wgt = {0: class_0_wgt, 1: class_1_wgt}

print(f"class 0 weight: {class_0_wgt:.2f}")
print(f"class 1 weight: {class_1_wgt:.2f}")

#### Build the model

In [ ]:
# build the CNN model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
model = Sequential()

# add Multiple layers of convolutional, pooling, and dense (fully-connected) layers with batch normalization
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(180, 180, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(180, 180, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

#### Compile the model

In [ ]:
# compile the model using below parameters
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

#### Train the model

In [ ]:
# define a function for repeating validation data infinitely
def repeat_generator(generator):
    while True:
        for batch in generator:
            yield batch

# Create a validation data generator that infinitely repeats the validation data
validation_generator = repeat_generator(validation)

# Fit the model using the validation data generator
cnn_model = model.fit(
    train, 
    epochs=10,
    validation_data=validation_generator, 
    class_weight=class_wgt,
    steps_per_epoch=100,
    validation_steps=25,
    verbose=1,  # Add this argument to see the progress of the training 
)

#### Plot training metrics

In [ ]:
# Plot the epochs vs loss graph of training and validation data
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(cnn_model.history['loss'], 'g-o', label='Training loss')
plt.plot(cnn_model.history['val_loss'], 'r-o', label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Loss Evaluation')

In [ ]:
# Plot the epochs vs accuracy graph of training and validation data
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 2)
plt.plot(cnn_model.history['accuracy'], 'go-', label='Training accuracy')
plt.plot(cnn_model.history['val_accuracy'], 'ro-', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper right')
plt.title('Accuracy Evaluation')

#### Evaluate the Model

In [ ]:
# evaluate the model with test data
test_acc = model.evaluate(test)
# calculate accuarcy
print(f"Test Accuracy: {test_acc[1] * 100:.2f}%")

# evaluate the model with train data
train_acc = model.evaluate(train)
# calculate accuarcy
print(f"Train Accuracy: {train_acc[1] * 100:.2f}%")

#### Testing

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# predict the output with test data using trained model
cnn_pred = model.predict(test)

#### Results

In [ ]:
# compute confusion matrix with threshold 0.5 and print
cm=confusion_matrix(test.classes, cnn_pred > 0.5)
print(cm)

# compute classification report with threshold 0.5 and print
report=classification_report(test.classes, cnn_pred > 0.5, output_dict=True)
df=pd.DataFrame(report)
print(df)

In [ ]:
# compute confusion matrix with threshold 0.7 and print
print(confusion_matrix(test.classes, cnn_pred > 0.7))

# compute classification report with threshold 0.7 and print
pd.DataFrame(classification_report(test.classes, cnn_pred > 0.7, output_dict=True))

## Transfer Learning - Existing Models

The pre-trained convolutional neural network (CNN) models DenseNet121, ResNet50, InceptionV3, InceptionResNetV2, MobileNetV2, and EfficientNetB0 have all been trained to categorize images using big datasets.

### Dense Network

#### Build the model

In [ ]:
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

# build the model using Densenet121 from keras
base_model = DenseNet121(input_shape=(180, 180, 3), include_top=False, weights='imagenet', pooling='avg')
base_model.summary()

In [ ]:
# print the number of layers in the model
layers = base_model.layers
print(f"The model has {len(layers)} layers")

In [ ]:
# print the input and output shapes
print(f"The input shape {base_model.input}")
print(f"The output shape {base_model.output}")

#### Compile the model

In [ ]:
# Initialize the model and add pooling layers
base_model = DenseNet121(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation="sigmoid")(x)
model = Model(inputs=base_model.input, outputs=predictions)

# compile the model
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

#### Train the model

In [ ]:
# define a function for repeating validation data infinitely
def repeat_generator(generator):
    while True:
        for batch in generator:
            yield batch

# Create a validation data generator that infinitely repeats the validation data
validation_generator = repeat_generator(validation)

# Fit the model using the validation data generator
densenet_model = model.fit(
    train, 
    epochs=10,
    validation_data=validation_generator,
    class_weight=class_wgt,
    steps_per_epoch=100,
    validation_steps=25,
)

#### Plot training metrics

In [ ]:
plt.figure(figsize=(12, 8))

# Plot the epochs vs loss graph of training and validation data
plt.subplot(2, 2, 1)
plt.plot(densenet_model.history['loss'], label='Loss')
plt.plot(densenet_model.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evaluation')

# Plot the epochs vs accuracy graph of training and validation data
plt.subplot(2, 2, 2)
plt.plot(densenet_model.history['accuracy'], label='Accuracy')
plt.plot(densenet_model.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evaluation')

#### Evaluate the Model

In [ ]:
# evaluate the model with test data
evaluation = model.evaluate(test)
# calculate accuarcy
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

# evaluate the model with test data
evaluation = model.evaluate(train)
# calculate accuarcy
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

#### Testing

In [ ]:
# predict the output with test data using trained model
pred = model.predict(test, steps=len(test))

#### Results

In [ ]:
# compute confusion matrix with threshold 0.5 and print
print(confusion_matrix(test.classes, pred > 0.5))
# compute classification report with threshold 0.5 and print
pd.DataFrame(classification_report(test.classes, pred > 0.5, output_dict=True))

### Residual Network

#### Build the model

In [ ]:
from keras.applications import ResNet50

# build the model using ResNet50 from keras
resnet_base_model = ResNet50(input_shape=(180,180,3), include_top=False, weights='imagenet')
resnet_base_model.summary()

In [ ]:
# add Multiple layers of pooling, dropout and dense (fully-connected) layers with batch normalization
resnet_model = Sequential([
        resnet_base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        BatchNormalization(),
        Dropout(0.6),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),
        Dense(64,activation="relu"),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1,activation="sigmoid")
    ])

#### Compile the model

In [ ]:
# set optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# define evaluation metrics
METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

# compile the model
resnet_model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=METRICS)

#### Train the model

In [ ]:
# define a function for repeating validation data infinitely
def repeat_generator(generator):
    while True:
        for batch in generator:
            yield batch

# Create a validation data generator that infinitely repeats the validation data
validation_generator = repeat_generator(validation)

# Fit the model using the validation data generator
rn_model = resnet_model.fit(train,
          epochs=10,
          validation_data=validation_generator,
          class_weight=class_wgt,
          steps_per_epoch=100,
          validation_steps=25)

#### Plot training metrics

In [ ]:
plt.figure(figsize=(12, 8))

# Plot the epochs vs loss graph of training and validation data
plt.subplot(2, 2, 1)
plt.plot(rn_model.history['loss'], label='Loss')
plt.plot(rn_model.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

# Plot the epochs vs accuracy graph of training and validation data
plt.subplot(2, 2, 2)
plt.plot(rn_model.history['accuracy'], label='Accuracy')
plt.plot(rn_model.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

#### Evaluate the Model

In [ ]:
# evaluate the model with test data
evaluation = resnet_model.evaluate(test)
# calculate accuracy
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

# evaluate the model with train data
evaluation = resnet_model.evaluate(train)
# calculate accuracy
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

#### Testing

In [ ]:
# predict the output with test data using trained model
pred = resnet_model.predict(test, steps=len(test))

#### Results

In [ ]:
# compute confusion matrix with threshold 0.5 and print
print(confusion_matrix(test.classes, pred > 0.5))
# compute classification report with threshold 0.5 and print
pd.DataFrame(classification_report(test.classes, pred > 0.5, output_dict=True))

### Inception Network

#### Build the model

In [ ]:
from keras.applications import InceptionV3

# build the model using InceptionV3 from keras
inception_base_model = InceptionV3(input_shape=(180,180,3),include_top=False,weights='imagenet')
inception_base_model.summary()

In [ ]:
# add Multiple layers of pooling, dropout and dense (fully-connected) layers with batch normalization
inception_model = Sequential([
        inception_base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        BatchNormalization(),
        Dropout(0.6),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),
        Dense(64,activation="relu"),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1,activation="sigmoid")
    ])

#### Compile the model

In [ ]:
# set optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# define evaluation metrics
METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

# compile the model
inception_model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=METRICS)

#### Train the model

In [ ]:
# define a function for repeating validation data infinitely
def repeat_generator(generator):
    while True:
        for batch in generator:
            yield batch

# Create a validation data generator that infinitely repeats the validation data
validation_generator = repeat_generator(validation)

# Fit the model using the validation data generator
im = inception_model.fit(train,
          epochs=10,
          validation_data=validation_generator,
          class_weight=class_wgt,
          steps_per_epoch=100,
          validation_steps=25)

#### Plot training metrics

In [ ]:
plt.figure(figsize=(12, 8))

# Plot the epochs vs loss graph of training and validation data
plt.subplot(2, 2, 1)
plt.plot(im.history['loss'], label='Loss')
plt.plot(im.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

# Plot the epochs vs accuracy graph of training and validation data
plt.subplot(2, 2, 2)
plt.plot(im.history['accuracy'], label='Accuracy')
plt.plot(im.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

#### Evaluate the Model

In [ ]:
# evaluate the model with test data
evaluation = inception_model.evaluate(test)
# calculate accuarcy
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

# evaluate the model with train data
evaluation = inception_model.evaluate(train)
# calculate accuarcy
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

#### Testing

In [ ]:
# predict the output with test data using trained model
pred = inception_model.predict(test, steps=len(test))

#### Results

In [ ]:
# compute confusion matrix with threshold 0.5 and print
print(confusion_matrix(test.classes, pred > 0.5))
# compute classification report with threshold 0.5 and print
pd.DataFrame(classification_report(test.classes, pred > 0.5, output_dict=True))

## Transfer Learning - Proposed Models

The proposed models for pneumonia detection using chest x-ray images combines three transfer learning models: InceptionResNetV2, MobileNetV2, and EfficientNetB0. The output from the final layers of each model is concatenated and passed through a fully connected layer with a sigmoid activation function, which produces a probability score for pneumonia classification.

### Inception-Residual Network

#### Build the model

In [ ]:
from keras.applications import InceptionResNetV2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout

# build the model using InceptionResNetV2 from keras
inception_resnet_base_model = InceptionResNetV2(input_shape=(180, 180, 3), include_top=False, weights='imagenet')
inception_base_model.summary()

In [ ]:
# add Multiple layers of pooling, dropout and dense (fully-connected) layers with batch normalization
inception_resnet_model = Sequential([
        inception_resnet_base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        BatchNormalization(),
        Dropout(0.6),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),
        Dense(64, activation="relu"),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1, activation="sigmoid")
    ])

#### Compile the model

In [ ]:
# set optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# define evaluation metrics
METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

#compile the model
inception_resnet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=METRICS)

#### Train the model

In [ ]:
# define a function for repeating validation data infinitely
def repeat_generator(generator):
    while True:
        for batch in generator:
            yield batch

# Create a validation data generator that infinitely repeats the validation data
validation_generator = repeat_generator(validation)

# Fit the model using the validation data generator
im_resnet = inception_resnet_model.fit(train,
          epochs=10,
          validation_data=validation_generator,
          class_weight=class_wgt,
          steps_per_epoch=100,
          validation_steps=25)

#### Plot training metrics

In [ ]:
plt.figure(figsize=(12, 8))

# Plot the epochs vs loss graph of training and validation data
plt.subplot(2, 2, 1)
plt.plot(im_resnet.history['loss'], label='Loss')
plt.plot(im_resnet.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

# Plot the epochs vs accuracy graph of training and validation data
plt.subplot(2, 2, 2)
plt.plot(im_resnet.history['accuracy'], label='Accuracy')
plt.plot(im_resnet.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

#### Evaluate the Model

In [ ]:
# evaluate the model with test data
evaluation =inception_resnet_model.evaluate(test)
# calculate accuracy
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

# evaluate the model with train data
evaluation = inception_resnet_model.evaluate(train)
# calculate accuracy
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

#### Testing

In [ ]:
# predict the output with test data using trained model
pred = inception_model.predict(test, steps=len(test))

#### Results

In [ ]:
# compute confusion matrix with threshold 0.5 and print
print(confusion_matrix(test.classes, pred > 0.5))
# compute classification report with threshold 0.5 and print
pd.DataFrame(classification_report(test.classes, pred > 0.5, output_dict=True))

### Efficient Network

#### Build the model

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout

# build the model using EfficientNetB0 from keras
efficientnet_base_model = EfficientNetB0(input_shape=(180, 180, 3), include_top=False, weights='imagenet')
inception_base_model.summary()

In [ ]:
# add Multiple layers of pooling, dropout and dense (fully-connected) layers with batch normalization
efficientnet_model = Sequential([
        efficientnet_base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        BatchNormalization(),
        Dropout(0.6),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),
        Dense(64, activation="relu"),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1, activation="sigmoid")
    ])

#### Compile the model

In [ ]:
# set optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# define evaluation metrics
METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

# compile the model
efficientnet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=METRICS)

#### Train the model

In [ ]:
# define a function for repeating validation data infinitely
def repeat_generator(generator):
    while True:
        for batch in generator:
            yield batch

# Create a validation data generator that infinitely repeats the validation data
validation_generator = repeat_generator(validation)

# Fit the model using the validation data generator
ef_model = efficientnet_model.fit(train,
          epochs=10,
          validation_data=validation_generator,
          class_weight=class_wgt,
          steps_per_epoch=100,
          validation_steps=25)

#### Plot training metrics

In [ ]:
plt.figure(figsize=(12, 8))

# Plot the epochs vs loss graph of training and validation data
plt.subplot(2, 2, 1)
plt.plot(ef_model.history['loss'], label='Loss')
plt.plot(ef_model.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

# Plot the epochs vs accuracy graph of training and validation data
plt.subplot(2, 2, 2)
plt.plot(ef_model.history['accuracy'], label='Accuracy')
plt.plot(ef_model.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

#### Evaluate the Model

In [ ]:
# evaluate the model with test data
evaluation =efficientnet_model.evaluate(test)
# calculate accuracy
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

# evaluate the model with train data
evaluation = efficientnet_model.evaluate(train)
# calculate accuracy
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

#### Testing

In [ ]:
# predict the output with test data using trained model
pred = inception_model.predict(test, steps=len(test))

#### Results

In [ ]:
# compute confusion matrix with threshold 0.5 and print
print(confusion_matrix(test.classes, pred > 0.5))
# compute classification report with threshold 0.5 and print
pd.DataFrame(classification_report(test.classes, pred > 0.5, output_dict=True))

### Mobile Network

#### Build the model

In [ ]:
from keras.applications import MobileNetV2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout

# build the model using MobileNetV2 from keras
mobilenetv2_base_model = MobileNetV2(input_shape=(180, 180, 3), include_top=False, weights='imagenet')
inception_base_model.summary()

In [ ]:
# add Multiple layers of pooling, dropout and dense (fully-connected) layers with batch normalization
mobilenetv2_model = Sequential([
        mobilenetv2_base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        BatchNormalization(),
        Dropout(0.6),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),
        Dense(64, activation="relu"),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1, activation="sigmoid")
    ])

#### Compile the model

In [ ]:
# set optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# define evaluation metrics
METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

# compile the model
mobilenetv2_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=METRICS)

In [ ]:
# define a function for repeating validation data infinitely
def repeat_generator(generator):
    while True:
        for batch in generator:
            yield batch

# Create a validation data generator that infinitely repeats the validation data
validation_generator = repeat_generator(validation)

# Fit the model using the validation data generator
mnv2_model = mobilenetv2_model.fit(train,
          epochs=10,
          validation_data=validation_generator,
          class_weight=class_wgt,
          steps_per_epoch=100,
          validation_steps=25)



In [ ]:
plt.figure(figsize=(12, 8))

# Plot the epochs vs loss graph of training and validation data
plt.subplot(2, 2, 1)
plt.plot(mnv2_model.history['loss'], label='Loss')
plt.plot(mnv2_model.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

# Plot the epochs vs accuracy graph of training and validation data
plt.subplot(2, 2, 2)
plt.plot(mnv2_model.history['accuracy'], label='Accuracy')
plt.plot(mnv2_model.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')


In [ ]:
# evaluate the model with test data
evaluation = mobilenetv2_model.evaluate(test)
# calculate accuracy
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

# evaluate the model with train data
evaluation = mobilenetv2_model.evaluate(train)
# calculate accuracy
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

#### Testing

In [ ]:
# predict the output with test data using trained model
pred = inception_model.predict(test, steps=len(test))

#### Results

In [ ]:
# compute confusion matrix with threshold 0.5 and print
print(confusion_matrix(test.classes, pred > 0.5))
# compute classification report with threshold 0.5 and print
pd.DataFrame(classification_report(test.classes, pred > 0.5, output_dict=True))

## Discussion

The results show that transfer learning models, particularly Inception-resnetv2, outperform the base CNN model in pneumonia diagnosis using chest X-ray images. This suggests that pre-trained models can be leveraged to improve the accuracy of medical image analysis tasks. Overall, the study provides promising results for the use of transfer learning models in pneumonia diagnosis using chest X-ray images, but further research is needed to address the limitations and improve the robustness of the method.